In [3]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

url = 'https://tomsk.hh.ru/search/vacancy'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
}
params = {
    'text': vacancy,
}


In [2]:
vacancy = input('Input  the vacancy your intereted in:\n')

Input  the vacancy your intereted in:
web


In [4]:
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

In [5]:
def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [6]:
res = []
params = {
    'text': vacancy,
}
while True:
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        break
    print(response.url)
    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      res.append(parse_vacancy_hh(vacancy))
    if 'page' in params.keys():
        params['page'] += 1
    else:
        params['page'] = 1

     

pprint(res)

https://tomsk.hh.ru/search/vacancy?text=web
https://tomsk.hh.ru/search/vacancy?text=web&page=1
https://tomsk.hh.ru/search/vacancy?text=web&page=2
https://tomsk.hh.ru/search/vacancy?text=web&page=3
https://tomsk.hh.ru/search/vacancy?text=web&page=4
https://tomsk.hh.ru/search/vacancy?text=web&page=5
https://tomsk.hh.ru/search/vacancy?text=web&page=6
https://tomsk.hh.ru/search/vacancy?text=web&page=7
https://tomsk.hh.ru/search/vacancy?text=web&page=8
https://tomsk.hh.ru/search/vacancy?text=web&page=9
https://tomsk.hh.ru/search/vacancy?text=web&page=10
https://tomsk.hh.ru/search/vacancy?text=web&page=11
https://tomsk.hh.ru/search/vacancy?text=web&page=12
https://tomsk.hh.ru/search/vacancy?text=web&page=13
https://tomsk.hh.ru/search/vacancy?text=web&page=14
https://tomsk.hh.ru/search/vacancy?text=web&page=15
https://tomsk.hh.ru/search/vacancy?text=web&page=16
https://tomsk.hh.ru/search/vacancy?text=web&page=17
https://tomsk.hh.ru/search/vacancy?text=web&page=18
https://tomsk.hh.ru/search/va

In [ ]:
!sudo apt install mongodb > log

In [ ]:
!service mongodb start

In [10]:
!pip install pymongo

     |████████████████████████████████| 519 kB 838 kB/s eta 0:00:01    |██                              | 30 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 2.0 MB/s eta 0:00:01


In [11]:
from pymongo import MongoClient
client = MongoClient()
db = client.vacancies_python_hh
collection_vacancies_hh_ru = db.hh_ru

In [12]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [14]:
cheak_and_save_vacancies_in_db(res)

In [35]:
result_find = list(collection_vacancies_hh_ru.find())
len(result_find)

772

In [57]:
salary = input('Input salary your intereted in:\n')

Input salary your intereted in:
200000


In [50]:
def find_salary_more(salary):
    return list(collection_vacancies_hh_ru.find({'$or':[{'max_salary':{'$gte': int(salary)}},{'min_salary':{'$gte': int(salary)}}]}))
#     return collection_vacancies_hh_ru.find({'max_salary':{'$gte': 100000}})

In [58]:
good_salary_vacancies = list(find_salary_more(salary))

In [59]:
pprint(good_salary_vacancies)

[{'_id': ObjectId('63f5ce70824abfe71f145375'),
  'currency_salary': 'руб.',
  'max_salary': 450000,
  'min_salary': 300000,
  'vacancy_link': 'https://tomsk.hh.ru/vacancy/72216524?from=vacancy_search_list&query=web',
  'vacancy_name': 'Python Backend Developer / WEB - разработчик',
  'vacancy_salary': '300\u202f000 – 450\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f5ce70824abfe71f145383'),
  'currency_salary': 'руб.',
  'max_salary': 200000,
  'min_salary': 200000,
  'vacancy_link': 'https://tomsk.hh.ru/vacancy/77224676?from=vacancy_search_list&query=web',
  'vacancy_name': 'Web-разработчик (bitrix)',
  'vacancy_salary': '200\u202f000 – 200\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('63f5ce70824abfe71f145384'),
  'currency_salary': 'KZT',
  'max_salary': 300000,
  'min_salary': None,
  'vacancy_link': 'https://tomsk.hh.ru/vacancy/76929486?from=vacancy_search_list&query=web',
  'vacancy_name': 'Junior UX/UI-дизайнер',
  'vacancy_salary': 'до 3

In [60]:
len(good_salary_vacancies)

82